In [1]:
#Imports
import pandas as pd 
import numpy as np
import pickle

from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn import metrics
from sklearn.model_selection import train_test_split

## Carga de datos

In [5]:
# Carga de datos etiquetados
nhanesDF = pd.read_csv('/Users/raul/Documents/UOC/TFM/NHANEScluster/Data/Processed/nhanesLabeled.csv',
                           sep=',', index_col=[0])
nhanesDF.shape

(2350, 26)

In [6]:
nhanesDF.head()

,SEQN,BMXBMI,BMXWAIST,BPXDI1,DIQ010,LBXGLU,LBXIN,HIQ011,HOD050,HOQ065,...,PAQ710,PAQ715,RHQ160,SLD012,SMQ040,RIAGENDR,RIDAGEYR,RIDRETH3,DMDEDUC2,CLUSTER
11,62172,33.3,120.4,70.0,2.0,104.0,18.62,1,4.0,2.0,...,5.0,5.0,3.0,8.0,1.0,2,43,4,3.0,1
38,62199,28.0,107.8,70.0,2.0,100.0,10.02,1,4.0,2.0,...,1.0,1.0,0.0,8.0,3.0,1,57,3,5.0,1
241,62402,23.3,82.0,70.0,2.0,107.0,7.81,2,6.0,3.0,...,4.0,0.5,0.0,9.0,1.0,1,48,6,1.0,1
250,62411,26.3,100.1,62.0,2.0,96.0,16.07,1,3.0,2.0,...,4.0,0.0,4.0,7.0,1.0,2,51,4,4.0,1
269,62430,37.9,114.2,74.0,2.0,97.0,15.90,1,5.0,2.0,...,3.0,3.0,0.0,6.0,3.0,1,27,2,5.0,1


## Preparación datos

In [10]:
# Split datos CV

X = nhanesDF.loc[:, 'BMXBMI':'DMDEDUC2']
y = nhanesDF['CLUSTER']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_test.shape, y_test.shape

((470, 24), (470,))

## Entrenamiento modelo regresión logística

In [12]:
lr = LogisticRegression(max_iter=10000)
lr.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=10000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

## Validación del modelo

In [13]:
#CrossValidation
from sklearn.model_selection import cross_val_score
scores = cross_val_score(lr, X, y, cv=5)
scores

array([0.99361702, 0.99148936, 0.99361702, 0.99148936, 0.98510638])

In [14]:
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.99 (+/- 0.01)


In [25]:
y_test.value_counts()

2    274
1    196
Name: CLUSTER, dtype: int64

In [15]:
y_pred=lr.predict(X_test)
c_matrix = metrics.confusion_matrix(y_test, y_pred)
c_matrix

array([[195,   1],
       [  5, 269]])

In [18]:
metrics.r2_score(y_test, y_pred)

0.9474899448830627

In [20]:
metrics.cohen_kappa_score(y_test, y_pred)

0.973821017452655

In [16]:
# Resultados Regresión conjunto de test
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print("Precision:",metrics.precision_score(y_test, y_pred))
print("Recall:",metrics.recall_score(y_test, y_pred))

Accuracy: 0.9872340425531915
Precision: 0.975
Recall: 0.9948979591836735


## Test carga del modelo previamente serializado.

In [48]:
# Test carga modelo a empotrar en aplicación web
model = pickle.load(open('model.pkl','rb'))

In [49]:
model

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)